In [4]:
import os
import tensorflow as tf

import numpy as np
import pachi_py

from src.game import TrainingEnvironment, PachiPlayer, roll_axis, get_legal_actions
from src.net import DualRes

In [12]:
# Load tensorflow graph
version = 'v1'
tf.reset_default_graph()
sess = tf.Session()
meta = os.path.join('models', version, version + '.meta')
saver = tf.train.import_meta_graph(meta)
saver.restore(sess, tf.train.latest_checkpoint(os.path.join('models', version)))
graph = tf.get_default_graph()
tf_board = graph.get_tensor_by_name('Placeholder:0')
tf_player = graph.get_tensor_by_name('Placeholder_1:0')
tf_pi = graph.get_tensor_by_name('policy_head/dense/BiasAdd:0')
tf_v = graph.get_tensor_by_name('value_head/Tanh:0')


def sample(board, player):
    pi, v = sess.run([tf_pi, tf_v], {tf_board:[board], tf_player:[player]})
    return pi, v

INFO:tensorflow:Restoring parameters from models/v1/v1


In [29]:
class TfPlayer():
    def __init__(self, colour, sample_fn):
        self.colour = colour
        self.c = colour - 1
        self.sample_fn = sample_fn
        
        self.i = 0

    def act(self, curr_state, *args):
        assert curr_state.color == self.colour
        print(curr_state)
        s = roll_axis(curr_state.board.encode())
        
        
        pi, v = self.sample_fn(s, self.c)
        pi = np.squeeze(pi)
        
        # allow only legal actions
        la = get_legal_actions(env.state.board, self.colour)
        pi = np.argsort(pi)[::-1]
        i = 0
        while pi[i] not in la:
            i+=1
        a = pi[i]
        print('v:', v, 'a:', a)
        
        self.i +=1
        return a

In [31]:
env = TrainingEnvironment(record = None)
# white_bot = PachiPlayer(pachi_py.WHITE, env.state.board)
# black_bot = TfPlayer(pachi_py.BLACK, sample)

white_bot = TfPlayer(pachi_py.WHITE, sample)
black_bot = PachiPlayer(pachi_py.BLACK, env.state.board)
env.set_players(black_bot, white_bot)
env.run()

To play: white
Move:   1  Komi: 0.0  Handicap: 0  Captures B: 0 W: 0
      A B C D E F G H J  
    +-------------------+
  9 | . . . . . . . . . |
  8 | . . . . . X). . . |
  7 | . . . . . . . . . |
  6 | . . . . . . . . . |
  5 | . . . . . . . . . |
  4 | . . . . . . . . . |
  3 | . . . . . . . . . |
  2 | . . . . . . . . . |
  1 | . . . . . . . . . |
    +-------------------+
v: [[-0.28645474]] a: 23
To play: white
Move:   3  Komi: 0.0  Handicap: 0  Captures B: 0 W: 0
      A B C D E F G H J  
    +-------------------+
  9 | . . . . . . . . . |
  8 | . . . . . X . . . |
  7 | . . . . X)O . . . |
  6 | . . . . . . . . . |
  5 | . . . . . . . . . |
  4 | . . . . . . . . . |
  3 | . . . . . . . . . |
  2 | . . . . . . . . . |
  1 | . . . . . . . . . |
    +-------------------+
v: [[-0.29643655]] a: 15
To play: white
Move:   5  Komi: 0.0  Handicap: 0  Captures B: 0 W: 0
      A B C D E F G H J  
    +-------------------+
  9 | . . . . . . . . . |
  8 | . . . . . X O . . |
  7 | . . . . X